<i>Copyright (c) Microsoft Corporation. All rights reserved.</i>

<i>Licensed under the MIT License.</i>

# Train SAR on MovieLens with Azure Machine Learning (Python, CPU)

This notebook provides an exmaple of how to train SAR on remote compute resources. Details of SAR algorithm can be found in [SAR Python CPU Movielens](https://github.com/Microsoft/Recommenders/blob/master/notebooks/00_quick_start/sar_movielens.ipynb) notebook. 

This notebook provides an example of how to utilize and evaluate SAR in Python on a CPU.

This notebook showcases what are the changes required to run local script on AzureML targets. AzureML comes with auto-scaling and gpu options and can greatly accelerate model training. There're a few more advanced AzureML notebooks in this repo. 

SAR is a fast scalable adaptive algorithm for personalized recommendations based on user transaction history. It produces easily explainable / interpretable recommendations and handles "cold item" and "semi-cold user" scenarios. SAR is a kind of neighborhood based algorithm (as discussed in [Recommender Systems by Aggarwal](https://dl.acm.org/citation.cfm?id=2931100)) which is intended for ranking top items for each user. 

SAR recommends items that are most ***similar*** to the ones that the user already has an existing ***affinity*** for. Two items are ***similar*** if the users who have interacted with one item are also likely to have interacted with another. A user has an ***affinity*** to an item if they have interacted with it in the past.

### Advantages of SAR:
- High accuracy for an easy to train and deploy algorithm
- Fast training, only requiring simple counting to construct matrices used at prediction time. 
- Fast scoring, only involving multiplication of the similarity matric with an affinity vector

### Advantages of using AML:
- Run large dataset quickly with GPU option
- Easy to scale out when dataset grows - by just creating and pointing to new compute target
- Parallelization on multiple nodes

### Notes to use SAR properly:
- Since it does not use item or user features, it can be at a disadvantage against algorithms that do.
- It's memory-hungry, requiring the creation of an $mxm$ sparse square matrix (where $m$ is the number of items). This can also be a problem for many matrix factorization algorithms.
- SAR favors an implicit rating scenario and it does not predict ratings.

# 0 Setup environment
### 1 Configure AzureML workspace
To get started on AzureML, you will need below items:
1. Azure subscription
   - To create one, Click "Start free" button [here](https://azure.microsoft.com/en-us/free/)
   - The account is free and comes with initial $200 credit for 30 days. This will easily cover small dataset experiments
2. AzureML workspace
   - To create one, follow below steps or see [this tutorial in more details](https://docs.microsoft.com/en-us/azure/machine-learning/service/setup-create-workspace#portal)
     1. Sign in to the [Azure portal](https://portal.azure.com) by using the credentials for the Azure subscription you use.
     2. Select **Create a resource** menu, search for **Machine Learning service workspace** select **Create** button
     3. In the **ML service workspace** pane, configure your workspace with entering the *workspace name* and *resource group* (or **create new** resource group if you don't have one already), and select **Create**. It can take a few moments to create the workspace.

Here's a quickstart video about [experiment with a local notebook] (https://docs.microsoft.com/en-us/azure/machine-learning/service/quickstart-run-local-notebook)

In [28]:
# set the environment path to find Recommenders
import sys
sys.path.append("../../")
import os
import shutil

from reco_utils.dataset import movielens

import azureml
from azureml.core import Workspace, Run, Experiment
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.train.estimator import Estimator
from azureml.widgets import RunDetails

### 2 Connect to AzureML workspace
Uncomment and run below cell if trying to connect to workspace for the first time in this environment. It will save `aml_config\config.json` which contains workspace configuration for future notebook runs. See [Workspace class doc](https://docs.microsoft.com/en-us/python/api/azureml-core/azureml.core.workspace.workspace?view=azure-ml-py).

In [ ]:
# %load ../../scripts/create_workspace.py

In [41]:
ws = Workspace.from_config()

Found the config file in: /data/home/cathack/notebooks/Recommenders/notebooks/00_quick_start/aml_config/config.json


### 3 Create Remote Compute Target
We create a cpu cluster as our **remote compute target**. If a cluster with the same name already exist in your workspace, the script will load it instead. You can read [Set up compute targets for model training](https://docs.microsoft.com/en-us/azure/machine-learning/service/how-to-set-up-training-targets) to learn more about setting up compute target on different locations.

According to [Pricing calculator](https://azure.microsoft.com/en-us/pricing/calculator/), East US VMs of size `STANDARD_D2_V2` with lowpriority setting comes at $0.112 per hour. Note that 10m and 20m dataset requires more capacity than `STANDARD_D2_V2`, such as `STANDARD_NC6` or `STANDARD_NC12`. See list of all available VM sizes [here](https://docs.microsoft.com/en-us/azure/templates/Microsoft.Compute/2018-10-01/virtualMachines?toc=%2Fen-us%2Fazure%2Fazure-resource-manager%2Ftoc.json&bc=%2Fen-us%2Fazure%2Fbread%2Ftoc.json#hardwareprofile-object).

In [42]:
# Remote compute (cluster) configuration. If you want to save the cost more, set these to small.
VM_SIZE = 'STANDARD_D2_V2'
VM_PRIORITY = 'lowpriority'
# Cluster nodes
MIN_NODES = 0
MAX_NODES = 2

CLUSTER_NAME = 'cpucluster'

try:
    compute_target = ComputeTarget(workspace=ws, name=CLUSTER_NAME)
    print("Found existing compute target")
except:
    print("Creating a new compute target...")
    compute_config = AmlCompute.provisioning_configuration(
        vm_size=VM_SIZE,
        vm_priority=VM_PRIORITY,
        min_nodes=MIN_NODES,
        max_nodes=MAX_NODES
    )
    # create the cluster
    compute_target = ComputeTarget.create(ws, CLUSTER_NAME, compute_config)
    compute_target.wait_for_completion(show_output=True, min_node_count=None, timeout_in_minutes=20)

# Use the 'status' property to get a detailed status for the current cluster. 
print(compute_target.status.serialize())

Found existing compute target
{'allocationState': 'Steady', 'allocationStateTransitionTime': '2019-03-28T22:31:55.471000+00:00', 'creationTime': '2019-03-28T22:29:18.683546+00:00', 'currentNodeCount': 4, 'errors': None, 'modifiedTime': '2019-03-28T22:30:53.325847+00:00', 'nodeStateCounts': {'idleNodeCount': 4, 'leavingNodeCount': 0, 'preemptedNodeCount': 0, 'preparingNodeCount': 0, 'runningNodeCount': 0, 'unusableNodeCount': 0}, 'provisioningState': 'Succeeded', 'provisioningStateTransitionTime': None, 'scaleSettings': {'minNodeCount': 4, 'maxNodeCount': 8, 'nodeIdleTimeBeforeScaleDown': 'PT120S'}, 'targetNodeCount': 4, 'vmPriority': 'LowPriority', 'vmSize': 'STANDARD_D2_V2'}


In [43]:
# top k items to recommend
TOP_K = 10

# Select Movielens data size: 100k, 1m, 10m, or 20m
MOVIELENS_DATA_SIZE = '1m'

### 3 Download dataset and upload to data store
Now make the data accessible remotely by uploading that data from your local machine into Azure so it can be accessed for remote training. The datastore is a convenient construct associated with your workspace for you to upload/download data, and interact with it from your remote compute targets. It is backed by Azure blob storage account.

The data files are uploaded into a directory named `data` at the root of the datastore. Here's detailed documentation about [datastore](https://docs.microsoft.com/en-us/azure/machine-learning/service/how-to-access-data).

In [44]:
DATA_DIR = 'aml_data'
TARGET_DIR = 'movielens'

os.makedirs(DATA_DIR, exist_ok=True)

# download dataset
data = movielens.load_pandas_df(
    size=MOVIELENS_DATA_SIZE,
    header=['UserId','MovieId','Rating','Timestamp']
)

# upload dataset to workspace datastore
data_file_name = "movielens_" + MOVIELENS_DATA_SIZE + "_data.pkl"
data.to_pickle(os.path.join(DATA_DIR, data_file_name))

ds = ws.get_default_datastore()

ds.upload(src_dir=DATA_DIR, target_path=TARGET_DIR, overwrite=True, show_progress=True)

$AZUREML_DATAREFERENCE_eb5c9bbb4e574f79bbd16f73a59d4649

# 1 Prepare training script
### Create a directory
Create a directory to deliver the necessary code from your computer to the remote resource.

In [45]:
SCRIPT_FOLDER = './movielens-sar'
os.makedirs(SCRIPT_FOLDER, exist_ok=True)

### Create a training script
To submit the job to the cluster, first create a training script. Run the following code to create the training script called `train.py` in the directory you just created. This training adds a regularization rate to the training algorithm, so produces a slightly different model than the local version.

In [46]:
%%writefile $SCRIPT_FOLDER/train.py

import argparse
import os
import numpy as np
import pandas as pd
import itertools
import logging
import time

from azureml.core import Run
from sklearn.externals import joblib

from reco_utils.dataset import movielens
from reco_utils.dataset.python_splitters import python_random_split
from reco_utils.evaluation.python_evaluation import map_at_k, ndcg_at_k, precision_at_k, recall_at_k
from reco_utils.recommender.sar.sar_singlenode import SARSingleNode

TARGET_DIR = 'movielens'
OUTPUT_FILE_NAME = 'outputs/movielens_sar_model.pkl'
MODEL_FILE_NAME = 'tmp/movielens_sar_model.pkl'


# get hold of the current run
run = Run.get_context()

# let user feed in 2 parameters, the location of the data files (from datastore), and the regularization rate of the logistic regression model
parser = argparse.ArgumentParser()
parser.add_argument('--data-folder', type=str, dest='data_folder', help='data folder mounting point')
parser.add_argument('--data-file', type=str, dest='data_file', help='data file name')
parser.add_argument('--top-k', type=int, dest='top_k', default=10, help='top k items to recommend')
parser.add_argument('--data-size', type=str, dest='data_size', default=10, help='Movielens data size: 100k, 1m, 10m, or 20m')
args = parser.parse_args()

data_pickle_path = os.path.join(args.data_folder, args.data_file)

data = pd.read_pickle(path=data_pickle_path)

train, test = python_random_split(data)

# instantiate the SAR algorithm and set the index
header = {
    "col_user": "UserId",
    "col_item": "MovieId",
    "col_rating": "Rating",
    "col_timestamp": "Timestamp",
}

logging.basicConfig(level=logging.DEBUG, 
                    format='%(asctime)s %(levelname)-8s %(message)s')

model = SARSingleNode(
    remove_seen=True, similarity_type="jaccard", 
    time_decay_coefficient=30, time_now=None, timedecay_formula=True, **header
)

# train the SAR model
start_time = time.time()

model.fit(train)

train_time = time.time() - start_time
run.log(name="Training time", value=train_time)

start_time = time.time()

top_k = model.recommend_k_items(test)

test_time = time.time() - start_time
run.log(name="Prediction time", value=test_time)

# TODO: remove this call when the model returns same type as input
top_k['UserId'] = pd.to_numeric(top_k['UserId'])
top_k['MovieId'] = pd.to_numeric(top_k['MovieId'])

# evaluate
eval_map = map_at_k(test, top_k, col_user="UserId", col_item="MovieId", 
                    col_rating="Rating", col_prediction="prediction", 
                    relevancy_method="top_k", k=args.top_k)
eval_ndcg = ndcg_at_k(test, top_k, col_user="UserId", col_item="MovieId", 
                      col_rating="Rating", col_prediction="prediction", 
                      relevancy_method="top_k", k=args.top_k)
eval_precision = precision_at_k(test, top_k, col_user="UserId", col_item="MovieId", 
                                col_rating="Rating", col_prediction="prediction", 
                                relevancy_method="top_k", k=args.top_k)
eval_recall = recall_at_k(test, top_k, col_user="UserId", col_item="MovieId", 
                          col_rating="Rating", col_prediction="prediction", 
                          relevancy_method="top_k", k=args.top_k)

run.log("map", eval_map)
run.log("ndcg", eval_ndcg)
run.log("precision", eval_precision)
run.log("recall", eval_recall)

os.makedirs('outputs', exist_ok=True)
os.makedirs('tmp', exist_ok=True)
# automatic upload of everything in /output folder doesn't work for large dataset
# model file has to be saved to a temp location, then uploaded by upload_file function
joblib.dump(value=model, filename=MODEL_FILE_NAME)

run.upload_file(OUTPUT_FILE_NAME, MODEL_FILE_NAME)

Overwriting ./movielens-sar/train.py


In [57]:
# copy dependent python files
UTILS_FOLDER = './movielens-sar/reco_utils/'
shutil.rmtree(UTILS_FOLDER, ignore_errors=True)
shutil.copytree('../../reco_utils/', UTILS_FOLDER)

'./movielens-sar/reco_utils/'

# 2 Run training script
### Create an estimator
An estimator object is used to submit the run.  Create your estimator by running the following code to define:

* The name of the estimator object, `est`
* The directory that contains your scripts. All the files in this directory are uploaded into the cluster nodes for execution. 
* The compute target.  In this case you will use the AmlCompute you created
* The training script name, train.py
* Parameters required from the training script 
* Python packages needed for training

In this tutorial, this target is AmlCompute. All files in the script folder are uploaded into the cluster nodes for execution. `ds.as_mount()` mounts a datastore on the remote compute and returns the folder. See documentation [here](https://docs.microsoft.com/en-us/azure/machine-learning/service/how-to-access-data#access-datastores-during-training).

In [48]:
script_params = {
    '--data-folder': ds.as_mount(),
    '--data-file': 'movielens/' + data_file_name,
    '--top-k': TOP_K,
    '--data-size': MOVIELENS_DATA_SIZE
}

est = Estimator(source_directory=SCRIPT_FOLDER,
                script_params=script_params,
                compute_target=compute_target,
                entry_script='train.py',
                conda_packages=['pandas'],
                pip_packages=['sklearn'])

### Submit the job to the cluster
Run the experiment by submitting the estimator object. You can check the status of current experiment in Azure Portal by clicking link below.

In [ ]:
# create experiment
EXPERIMENT_NAME = 'movielens-sar'
exp = Experiment(workspace=ws, name=EXPERIMENT_NAME)

run = exp.submit(config=est)
run

Above cell should output similar table as below
![Experiment submit output](../../images/output.jpg)
After clicking "Link to Azure Portal", experiment run details tab looks like this
![Azure Portal Experiment](../../images/workspace.jpg)


# 3 Monitor remote run

### Jupyter widget

Alternatively, watch the progress of the run with a Jupyter widget.  Like the run submission, the widget is asynchronous and provides live updates every 10-15 seconds until the job completes.

In [50]:
RunDetails(run).show()

_UserRunWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'NOTSET',…